In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [20]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
from dotenv import load_dotenv

load_dotenv("/content/drive/MyDrive/book-recommender/.env")

True

In [22]:
import os
os.getcwd()


'/content'

In [27]:
import os
os.listdir("/content/drive/MyDrive")


['https:  makecode.gdoc',
 'tt2139881-HCM-720.mkv',
 'CV- chebila-Faster-2019.pdf',
 'Motivation letter-Faster.pdf',
 'Adobe Photoshop CS6.zip',
 'الرد على انتشار الإسلام بالقوة والإجبار.pptx',
 'Amira_Ashraf_Arabic_teacher_from_Egypt_FLTA_program.mp4',
 'An_LLM_in_the_United_States_Advice_from_a_Fulbright_Fellow.mp4',
 'Anna_Kharchenko_Fulbrighter_from_Ukraine_Psychotherapist.mp4',
 'Fulbright Webinar – Tips for Writing a Superior Scholarship Application.mp4',
 'Fulbright_Interview_tips.mp4',
 'Norton_Rose_Fulbright_offers_applications_and_interview_tips_to_applicants.mp4',
 'Seeking a Masters Degree in the US - Tips from Fulbright Grantee Tom Pellens.mp4',
 'y2mate.com - a_masters_degree_in_the_united_states_3ZJqK388NKU_360p.mp4',
 'y2mate.com - desiree_daring_discusses_her_fulbright_experience_during_the_inaugural_fulbright_noir_seminar_TQFK43Oqwnc_1080p.mp4',
 'y2mate.com - being_a_fulbright_english_teaching_assistant_in_belgium_meghan_briggs_Hqc_UgUIHVM_1080p.mp4',
 'y2mate.com 

In [29]:
import pandas as pd

books = pd.read_csv("/content/drive/MyDrive/book-recommender/books_cleaned.csv")


In [30]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that:Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [5]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: str

In [31]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False,
                                   encoding="utf-8")

In [32]:
raw_documents = TextLoader("tagged_description.txt",
                           encoding="utf-8").load()



In [33]:
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [7]:
print(f"Total chunks: {len(documents)}")

Total chunks: 1587


In [38]:
db_books = Chroma.from_documents(
    documents,
    OpenAIEmbeddings()
)


In [47]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 2)

In [48]:
docs

[Document(id='eb4d5e84-8e39-4c89-8fac-dfefefeb0f3e', metadata={'source': 'tagged_description.txt'}, page_content="9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.\n9780786808373 Introducing your baby to birds, cats, dogs, and babies through fine art, illsutration and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing baby to some basic -- and sometimes playful -- information on the subjects."),
 Document(id='b454f45e-498f-4ef2-9560-a51be2c0c753', metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine ar

In [45]:
import numpy as np

In [50]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = top_k)
    books_list = []
    for i in range(len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)
    

In [51]:
retrieve_semantic_recommendations("A book about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
1479,9780345434906,0345434900,The Real James Herriot,James Wight;Jim Wight,Biography & Autobiography,http://books.google.com/books/content?id=TrOoi...,A portrait of the best-selling author of All C...,2001.0,4.20,371.0,4178.0,The Real James Herriot:A Memoir of My Father,9780345434906 A portrait of the best-selling a...
1641,9780374517885,0374517886,Sammlung,Isaac Bashevis Singer,Fiction,http://books.google.com/books/content?id=B0_ws...,The Noble laureate has selected nearly 150 of ...,1982.0,4.37,624.0,1603.0,Sammlung,9780374517885 The Noble laureate has selected ...
1696,9780375713934,037571393X,John James Audubon,Richard Rhodes,Art,http://books.google.com/books/content?id=4j2FD...,A biography of artist John James Audubon offer...,2006.0,4.14,528.0,437.0,John James Audubon:The Making of an American,9780375713934 A biography of artist John James...
1905,9780393311037,0393311031,Hen's Teeth and Horse's Toes,Stephen Jay Gould,Nature,http://books.google.com/books/content?id=EPh9j...,A collection of essays answers questions about...,1994.0,4.10,416.0,1680.0,Hen's Teeth and Horse's Toes,9780393311037 A collection of essays answers q...
2261,9780446518628,044651862X,The Celestine Prophecy,James Redfield,Fiction,http://books.google.com/books/content?id=UXolx...,You have never read a book like this before --...,1994.0,3.63,247.0,1280.0,The Celestine Prophecy:An Adventure,9780446518628 You have never read a book like ...
2409,9780451456816,0451456815,Not of Woman Born,Constance Ash;Robert A. Silverberg,Fiction,http://books.google.com/books/content?id=nOEIA...,A collection of stories that detail modern sci...,1999.0,3.80,288.0,59.0,Not of Woman Born,9780451456816 A collection of stories that det...
3676,9780767908184,076790818X,A Short History of Nearly Everything,Bill Bryson,Science,http://books.google.com/books/content?id=YjAnf...,The author of A Walk in the Woods traces the B...,2004.0,4.20,544.0,228522.0,A Short History of Nearly Everything,9780767908184 The author of A Walk in the Wood...
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...
4185,9780878937202,087893720X,Essentials of Conservation Biology,Richard B. Primack,Science,http://books.google.com/books/content?id=spaGQ...,Essentials of Conservation Biology (Fourth Edi...,2006.0,3.89,585.0,36.0,Essentials of Conservation Biology,9780878937202 Essentials of Conservation Biolo...
4359,9781400051533,1400051533,"The Hedgehog, the Fox, and the Magister's Pox",Stephen Jay Gould,Science,http://books.google.com/books/content?id=4f8v9...,Draws on the philosophy of seventh century B.C...,2003.0,3.64,288.0,387.0,"The Hedgehog, the Fox, and the Magister's Pox:...",9781400051533 Draws on the philosophy of seven...


In [ ]:
## Next stop 1:11:50